In [3]:
library(tidyverse)

Before running these scripts:
1. Set up MIMIC as a Postgres database.
2. Build the concept materialized views: https://github.com/MIT-LCP/mimic-code/tree/master/concepts
3. Build Sepsis3 materialized views: https://github.com/alistairewj/sepsis3-mimic/tree/master/query
4. Run the `get_data.sh` script in `sql` folder.

This script is drived work from https://github.com/spfohl/cs238_sepsis_rl and an attempt to reproduce the dataset for https://arxiv.org/abs/1705.08422 

In [4]:
data_path <- '../cs238_sepsis_rl/data/'

# Build Actions

In [6]:
cohort_df <- read_csv(paste0(data_path, 'cohort.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  suspected_infection_time_poe = col_datetime(format = ""),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  hospital_expire_flag = col_integer()
)


In [72]:
cohort_df  %>% group_by(subject_id)  %>% nrow()

[1] 18345

Firs discrepency between our approach and Raghu et. al. They have 17,898 patients.

## Build Time Intervals

In [6]:
get_time_intervals <- function(start, end) {
    interval_times <- seq(start, end, by = '4 hour')
    interval_start_time <- interval_times[1:length(interval_times) -1]
    interval_end_time <- interval_times[2:length(interval_times)]
    return(data.frame(interval_start_time, interval_end_time))
}

In [7]:
interval_times_df <- cohort_df %>% 
                        group_by(subject_id, hadm_id, icustay_id) %>% 
                        do(get_time_intervals(.$window_start, .$window_end))

In [8]:
dim(interval_times_df)

[1] 330210      5

## Vassopressors

In [9]:
# carevu
vassopressor_cv_df <- read_csv(paste0(data_path, 'vassopressors_cv_cohort.csv'))
# metavision
vassopressor_mv_df <- read_csv(paste0(data_path, 'vassopressors_mv_cohort.csv'))

Parsed with column specification:
cols(
  icustay_id = col_integer(),
  charttime = col_datetime(format = ""),
  itemid = col_integer(),
  stopped = col_character(),
  rate = col_double(),
  amount = col_double()
)
Parsed with column specification:
cols(
  icustay_id = col_integer(),
  linkorderid = col_integer(),
  starttime = col_datetime(format = ""),
  endtime = col_datetime(format = ""),
  itemid = col_integer(),
  rate = col_double(),
  amount = col_double()
)


In [12]:
# calculate max values for each interval
max_vassopressor_cv <- interval_times_df %>% 
                                left_join(vassopressor_cv_df) %>% 
                                group_by(subject_id, 
                                       hadm_id, 
                                       icustay_id, 
                                       interval_start_time, 
                                       interval_end_time) %>%
                                filter(charttime > interval_start_time & charttime <= interval_end_time) %>%
                                summarise(max_amount = max(amount))

max_vassopressor_mv <- interval_times_df %>% 
                                left_join(vassopressor_mv_df) %>% 
                                group_by(subject_id, 
                                           hadm_id, 
                                           icustay_id, 
                                           interval_start_time, 
                                           interval_end_time) %>%
                                filter(endtime > interval_start_time & endtime <= interval_end_time) %>%
                                summarise(max_amount = max(amount)) 

Joining, by = "icustay_id"
Joining, by = "icustay_id"


In [44]:
# Merge the carevue and metavision data
temp <- max_vassopressor_cv %>% 
                        full_join(max_vassopressor_mv) %>%
                        filter(!is.na(max_amount))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time", "max_amount")


### Action Quantization

In [57]:
# A function to map the data to quartiles
num2quartile <- function(x) {
    x_ <- x[x>0]
    quants <- quantile(x_, probs=0:4/4, na.rm = TRUE)
    print(quants)
    return(as.integer(cut(x, quants, include.lowest=TRUE)))
}

In [49]:
head(temp)

subject_id,hadm_id,icustay_id,interval_start_time,interval_end_time,max_amount
94,183686,229012,2176-02-26 00:30:00,2176-02-26 04:30:00,6.3000006
94,183686,229012,2176-02-26 04:30:00,2176-02-26 08:30:00,0.8000001
94,183686,229012,2176-02-26 08:30:00,2176-02-26 12:30:00,0.0000000
94,183686,229012,2176-02-26 12:30:00,2176-02-26 16:30:00,0.0000000
202,108295,228132,2145-10-23 05:38:00,2145-10-23 09:38:00,0.0000000
275,129886,219649,2170-10-08 06:30:00,2170-10-08 10:30:00,0.0000000


In [58]:
temp$discrete_pressor <- num2quartile(temp$max_amount)  %>% replace_na(0)

          0%          25%          50%          75%         100% 
2.840102e-03 1.035571e+00 3.550489e+00 1.231740e+01 7.952001e+02 


In [59]:
head(temp)

subject_id,hadm_id,icustay_id,interval_start_time,interval_end_time,max_amount,discrete_pressor
94,183686,229012,2176-02-26 00:30:00,2176-02-26 04:30:00,6.3000006,3
94,183686,229012,2176-02-26 04:30:00,2176-02-26 08:30:00,0.8000001,1
94,183686,229012,2176-02-26 08:30:00,2176-02-26 12:30:00,0.0000000,0
94,183686,229012,2176-02-26 12:30:00,2176-02-26 16:30:00,0.0000000,0
202,108295,228132,2145-10-23 05:38:00,2145-10-23 09:38:00,0.0000000,0
275,129886,219649,2170-10-08 06:30:00,2170-10-08 10:30:00,0.0000000,0


In [61]:
temp  %>% group_by(discrete_pressor)  %>% summarise(n=n())

discrete_pressor,n
0,680
1,3798
2,3798
3,3797
4,3798


In [67]:
sum(temp$max_amount>0)
sum(temp$discrete_pressor>0)

[1] 15191

[1] 15191

In [68]:
# join with all interval times
action_df_VP <- temp %>% right_join(interval_times_df) %>% replace_na(list(discrete_pressor = 0))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [70]:
action_df_VP  %>%  group_by(discrete_pressor)  %>% summarise(n=n())

discrete_pressor,n
0,315019
1,3798
2,3798
3,3797
4,3798


Values per sepsisrl repo: https://github.com/aniruddhraghu/sepsisrl/blob/master/preprocessing/process_interventions.ipynb
```
0.0    199294
4.0     12453
2.0     11215
1.0      9892
3.0      9596
```
There is a clear discrepency here...

In [27]:
action_df_VP <- temp %>% right_join(interval_times_df) %>% replace_na(list(discrete_pressor = 0))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [74]:
temp  %>%  group_by(discrete_pressor)  %>% summarise(n=n())

discrete_pressor,n
0,680
1,3798
2,3798
3,3797
4,3798


##  IV Fluids

In [ ]:
inputevents_cv_df <- read_csv(paste0(data_path, 'inputevents_cv_cohort.csv'))
inputevents_mv_df <- read_csv(paste0(data_path, 'inputevents_mv_cohort.csv'))

In [76]:
# get the ML values that are >0
ie_filt_cv <- inputevents_cv_df %>% 
                filter(amountuom == 'ml' & !is.na(amount) & amount >= 0)
ie_filt_mv <- inputevents_mv_df %>% 
                filter(amountuom == 'ml' & !is.na(amount) & amount > 0)

In [78]:
# Pulling out some data.table stuff to go FAST
library(data.table)

ie_filt_cv_dt <- as.data.table(ie_filt_cv, key = 'icustay_id')
ie_filt_mv_dt <- as.data.table(ie_filt_mv, key = 'icustay_id')
interval_times_dt <- as.data.table(interval_times_df, key = 'icustay_id')


# get values from Carevue and Metavision and merge
total_IV_cv <- interval_times_dt %>% 
                                merge(ie_filt_cv_dt, allow.cartesian = TRUE) %>% 
                                group_by(subject_id,
                                       hadm_id, 
                                       icustay_id, 
                                       interval_start_time, 
                                       interval_end_time) %>%
                                filter(charttime > interval_start_time & charttime <= interval_end_time) %>%
                                summarise(total_amount = sum(amount))

total_IV_mv <- interval_times_dt %>% 
                                merge(ie_filt_mv_dt, allow.cartesian = TRUE) %>% 
                                group_by(subject_id,
                                       hadm_id, 
                                       icustay_id, 
                                       interval_start_time, 
                                       interval_end_time) %>%
                                filter(endtime > interval_start_time & endtime <= interval_end_time) %>%
                                summarise(total_amount = sum(amount))

# Map to discrete vassopressor states
temp <- total_IV_cv %>% 
                        full_join(total_IV_mv) %>%
                        filter(!is.na(total_amount))

temp$discrete_IV <- num2quartile(temp$total_amount)
action_df_IV <- temp %>% right_join(interval_times_df) %>% replace_na(list(discrete_IV = 0))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time", "total_amount")


          0%          25%          50%          75%         100% 
1.666667e-02 2.192000e+02 4.777500e+02 1.000000e+03 2.901451e+05 


Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [79]:
group_by(action_df_IV, discrete_IV)  %>%  summarise(n=n())

discrete_IV,n
0,184118
1,36532
2,36533
3,36702
4,36360


To be compared to Raghu et al:

```
0.0    55819
2.0    48527
3.0    46658
4.0    46658
1.0    44788
```

# 🤷🏽‍♂️

In [82]:
# merge everything
action_df <- action_df_IV %>% full_join(action_df_VP)
action_df <- action_df %>% rename(total_IV = total_amount, max_VP = max_amount)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [84]:
# map to discrete actions across both categories
action_df <- action_df %>% mutate(discrete_action =discrete_pressor  + discrete_IV * 5)

In [85]:
# write files out to disk
write_csv(action_df, paste0(data_path, 'action_df.csv'))
write_csv(interval_times_df, paste0(data_path, 'interval_times_df.csv'))

# Build States

Now that we have actions, let's create states.

## Vitals

In [ ]:
# Load the vitals
vitals_df <- read_csv(paste0(data_path, 'vitals_cohort.csv'))

In [87]:
# Cast to DT
vitals_dt <- as.data.table(vitals_df, key = 'icustay_id')
interval_times_dt <- as.data.table(interval_times_df, key = 'icustay_id')

In [88]:
# merge to interval times
merged_vitals <- interval_times_dt %>% 
                    merge(vitals_dt, allow.cartesian = TRUE)

In [89]:
# average values per interval times
mean_vitals <- merged_vitals[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_vital = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          vital_id)]

## Labs

In [90]:
labs_df <- read_csv(paste0(data_path, 'labs_cohort.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  window_start = col_datetime(format = ""),
  window_end = col_datetime(format = ""),
  intime = col_datetime(format = ""),
  outtime = col_datetime(format = ""),
  charttime = col_datetime(format = ""),
  lab_id = col_character(),
  valuenum = col_double()
)


In [91]:
# cast to data.table to be fast!
labs_dt <- as.data.table(labs_df, key = 'icustay_id')
merged_labs <- interval_times_dt %>% 
                    merge(labs_dt, allow.cartesian = TRUE)

In [93]:
# calculate mean

mean_labs <- merged_labs[charttime > interval_start_time & charttime <= interval_end_time,
                                   .(mean_lab = mean(valuenum)), by = .(subject_id, 
                                                                          hadm_id, 
                                                                          icustay_id, 
                                                                          interval_start_time, 
                                                                          interval_end_time, 
                                                                          lab_id)]

In [94]:
mean_labs <- mean_labs %>% rename(meas_id = lab_id, mean_value = mean_lab)
mean_vitals <- mean_vitals %>% rename(meas_id = vital_id, mean_value = mean_vital)

# put vitals and labs together
mean_labs_vitals <- mean_labs %>% full_join(mean_vitals)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time", "meas_id", "mean_value")


In [95]:
vitals_labs_spread <- mean_labs_vitals %>% 
                        spread(meas_id, mean_value) %>% 
                        right_join(interval_times_dt) 

# Exclude times in which no measurements were made
vitals_labs_spread_filt <- vitals_labs_spread %>%
                                gather(lab_id, meas_value, ALBUMIN:WBC) %>%
                                group_by(subject_id, hadm_id, icustay_id, interval_start_time, interval_end_time) %>%
                                summarise(exclude = all(is.na(meas_value))) %>%
                                full_join(vitals_labs_spread) %>%
                                filter(!exclude)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")


In [96]:
# write out the vitals and labs
write_csv(vitals_labs_spread_filt, paste0(data_path, 'vitals_labs_spread_filt.csv'))

## Perform LVCF

In [100]:
vitals_labs <- vitals_labs_spread_filt
action_df <- read_csv(paste0(data_path, 'action_df.csv'))

Parsed with column specification:
cols(
  subject_id = col_integer(),
  hadm_id = col_integer(),
  icustay_id = col_integer(),
  interval_start_time = col_datetime(format = ""),
  interval_end_time = col_datetime(format = ""),
  total_IV = col_double(),
  discrete_IV = col_integer(),
  max_VP = col_double(),
  discrete_pressor = col_integer(),
  discrete_action = col_integer()
)


In [99]:
# Perform Last Value Carried Forward
vitals_labs_lvcf <- vitals_labs %>% group_by(subject_id, hadm_id, icustay_id) %>%
                        arrange(subject_id, hadm_id, icustay_id, interval_start_time) %>%
                        fill(ALBUMIN:WBC)

In [101]:
# merge demographics data
demographics <- read_csv(paste0(data_path, 'demographics_cohort.csv'))
data_all <- vitals_labs_lvcf %>% full_join(demographics) %>% ungroup()

Parsed with column specification:
cols(
  .default = col_integer(),
  age = col_double(),
  height = col_double(),
  weight = col_double()
)
See spec(...) for full column specifications.
Joining, by = c("subject_id", "hadm_id", "icustay_id")


# Train-Test Breakdown

To match Raghu et al, will not create a validation dataset. Only creating train/test (80/20).

In [102]:
# Generate the train/test split
set.seed(10)
ids <- unique(data_all$icustay_id)

train_prop <- 0.8
num_train <- floor(train_prop * length(ids))

train_ids <- sample(ids, num_train)
test_ids <- setdiff(ids, train_ids)

train_id_df <- data.frame(icustay_id = train_ids)
test_id_df <- data.frame(icustay_id = test_ids)

In [103]:
train_data <- data_all %>% inner_join(train_id_df) %>% ungroup()
test_data <- data_all %>% inner_join(test_id_df) %>% ungroup()

Joining, by = "icustay_id"
Joining, by = "icustay_id"


In [104]:
dim(train_data)
dim(test_data)

[1] 154538     52

[1] 39464    52

## Impute & Normalize

Use `caret` to normalize, and median impute.

In [108]:
library(caret)

In [106]:
feature_names <- setdiff(c(names(vitals_labs_lvcf), names(demographics)),
                         c('subject_id', 
                           'hadm_id', 
                           'icustay_id', 
                           'interval_start_time',
                           'interval_end_time',
                           'exclude'))
train_features <- train_data[, feature_names]
test_features <- test_data[, feature_names]

In [107]:
# Preprocess the data, keep a non-scaled version for future
preprocessor_no_scale <- preProcess(train_features, method = c('medianImpute'))
preprocessor <- preProcess(train_features, method = c('center', 'scale', 'medianImpute'))

train_proc <- predict(preprocessor, train_features)
test_proc <- predict(preprocessor, test_features)
test_no_scale <- predict(preprocessor_no_scale, test_features)

In [111]:
id_vars <- c('subject_id', 
               'hadm_id', 
               'icustay_id', 
               'interval_start_time',
               'interval_end_time')

train_out <- cbind(train_data[, !(names(train_data) %in% names(train_proc))], train_proc)
test_out <- cbind(test_data[, !(names(test_data) %in% names(test_proc))], test_proc)
test_no_scale_out <- cbind(test_data[, !(names(test_data) %in% names(test_no_scale))], test_no_scale)

In [114]:
dim(test_no_scale_out)

[1] 39464    52

In [115]:
dim(test_out)

[1] 39464    52

In [116]:
dim(train_out)

[1] 154538     52

In [118]:
write_csv(train_out, paste0(data_path, 'train_data.csv'))
write_csv(test_out, paste0(data_path, 'test_data.csv'))
write_csv(test_no_scale_out, paste0(data_path, 'test_no_scale_data.csv'))

# Build Rewards

Reward +15 if the patient is released otherwise -15

In [119]:
outcome_df <- cohort_df %>% group_by(subject_id, 
                                     hadm_id, 
                                     icustay_id)

In [121]:
train_df_action <- train_out %>% inner_join(select(action_df, subject_id, 
                                                   hadm_id, 
                                                   icustay_id, 
                                                   interval_start_time, 
                                                   interval_end_time,
                                                   discrete_action)) %>%
                                inner_join(cohort_df)

test_df_action_no_scale <- test_no_scale_out %>% inner_join(select(action_df, subject_id, 
                                                   hadm_id, 
                                                   icustay_id, 
                                                   interval_start_time, 
                                                   interval_end_time,
                                                   discrete_action)) %>%
                                inner_join(cohort_df)




test_df_action <- test_out %>% inner_join(select(action_df, subject_id, 
                                                   hadm_id, 
                                                   icustay_id, 
                                                   interval_start_time, 
                                                   interval_end_time,
                                                   discrete_action)) %>%
                                inner_join(cohort_df)

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "interval_end_time")
Joining, by = c("subject_id", "hadm_id", "icustay_id")


In [129]:
temp_train <- train_df_action %>% group_by(subject_id, 
                             hadm_id, 
                             icustay_id,
                             hospital_expire_flag) %>%
                    summarise(interval_start_time = max(interval_start_time)) %>%
                    mutate(reward = ifelse(hospital_expire_flag, -15, 15))

train_df_action_reward <- train_df_action %>% 
                                left_join(temp_train) %>%
                                replace_na(list(reward = 0))

temp_test_no_scale <- test_df_action_no_scale %>% group_by(subject_id, 
                             hadm_id, 
                             icustay_id,
                             hospital_expire_flag) %>%
                    summarise(interval_start_time = max(interval_start_time)) %>%
                    mutate(reward = ifelse(hospital_expire_flag, -15, 15))


test_df_action_reward_no_scale <- test_df_action_no_scale %>% 
                                left_join(temp_test_no_scale) %>%
                                replace_na(list(reward = 0))

temp_test <- test_df_action %>% group_by(subject_id, 
                             hadm_id, 
                             icustay_id,
                             hospital_expire_flag) %>%
                    summarise(interval_start_time = max(interval_start_time)) %>%
                    mutate(reward = ifelse(hospital_expire_flag, -15, 15))

test_df_action_reward <- test_df_action %>% 
                                left_join(temp_test) %>%
                                replace_na(list(reward = 0))

Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "hospital_expire_flag")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "hospital_expire_flag")
Joining, by = c("subject_id", "hadm_id", "icustay_id", "interval_start_time", "hospital_expire_flag")


In [130]:
temp2_train <- train_df_action_reward %>% 
                                mutate(row_id = 1:nrow(train_df_action_reward) - 1) %>% 
                                arrange(subject_id, hadm_id, icustay_id, interval_start_time, row_id) %>%
                                group_by(subject_id, hadm_id, icustay_id) %>%
                                mutate(row_id_next = ifelse(row_id == max(row_id), row_id, row_id + 1))

temp2_test_no_scale <- test_df_action_reward_no_scale %>% 
                                mutate(row_id = 1:nrow(test_df_action_reward_no_scale) - 1) %>% 
                                arrange(subject_id, hadm_id, icustay_id, interval_start_time, row_id) %>%
                                group_by(subject_id, hadm_id, icustay_id) %>%
                                mutate(row_id_next = ifelse(row_id == max(row_id), row_id, row_id + 1))


temp2_test <- test_df_action_reward %>% 
                                mutate(row_id = 1:nrow(test_df_action_reward) - 1) %>% 
                                arrange(subject_id, hadm_id, icustay_id, interval_start_time, row_id) %>%
                                group_by(subject_id, hadm_id, icustay_id) %>%
                                mutate(row_id_next = ifelse(row_id == max(row_id), row_id, row_id + 1))


In [131]:
final_df_train <- temp2_train %>% select(
                             -exclude, 
                             -intime, 
                             -outtime, 
                             -window_start,
                             -window_end,
                             -suspected_infection_time_poe
                             )

final_df_test_no_scale <- temp2_test_no_scale %>% select(
                             -exclude, 
                             -intime, 
                             -outtime, 
                             -window_start,
                             -window_end,
                             -suspected_infection_time_poe
                             )



final_df_test <- temp2_test %>% select(
                             -exclude, 
                             -intime, 
                             -outtime, 
                             -window_start,
                             -window_end,
                             -suspected_infection_time_poe
                             )

In [133]:
write_csv(final_df_train, paste0(data_path, 'train_state_action_reward_df.csv'))
write_csv(final_df_test_no_scale, paste0(data_path, 'test_state_action_reward_df_no_scale.csv'))
write_csv(final_df_test, paste0(data_path, 'test_state_action_reward_df.csv'))